In [76]:
import pandas as pd
import matplotlib.pyplot as plt

In [77]:
agg = pd.read_csv('agg.csv', low_memory=False)
agg = agg.drop(['index'], axis = 1)
weather = pd.read_csv('weather2.csv')
weather = weather.drop(['Unnamed: 0'], axis = 1)
park = pd.read_csv('park.csv')
park['YEAR'] = [i.replace('-01-01 00:00:00', '') for i in park['YEAR']]
park['YEAR'] = park['YEAR'].astype('int')
park = park[park['YEAR'] >= 2014]
park = park.reset_index()
park = park.drop(['index'], axis = 1)

In [78]:
abb = {'KAN' : 'KC', 'FLA' : 'MIA', 'CHW' : 'CWS', 'WAS' : 'WSH'}
for i in range(len(weather)):
    for word, initial in abb.items():
        weather.at[i, 'Team 1'] = weather.at[i, 'Team 1'].replace(word, initial)
        weather.at[i, 'Team 2'] = weather.at[i, 'Team 2'].replace(word, initial)

In [79]:
mergeTemp = pd.merge(how = "left", left = weather, right = park, left_on = 'Venue', right_on = 'PARK NAME')
mergeTemp2 = pd.merge(agg, mergeTemp, how = "inner", left_on = ["next_game_date", "HomeTeam", "AwayTeam"], right_on = ["Date", "Team 1", "Team 2"])
mergeTemp3 = pd.merge(agg, mergeTemp, how = "inner", left_on = ["next_game_date", "HomeTeam", "AwayTeam"], right_on = ["Date", "Team 2", "Team 1"])
merge = mergeTemp2.append(mergeTemp3)

In [80]:
merge = merge.drop_duplicates(['Name', 'Game_date'], keep = False)
merge = merge.reset_index()
merge = merge.drop(['index'], axis = 1)

In [81]:
merge = merge[merge["HomeTeam"].notnull()]
merge = merge[merge["AwayTeam"].notnull()]
merge = merge[merge["next_game_date"].notnull()]
merge.shape[0]

16434

In [82]:
fix = merge[merge['Venue'].isna()]
fix.shape[0]

0

In [83]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

x, y = pd.DataFrame(merge, columns = ['Games_played_to_date', 'BA', 'PA', 'ISO', 'SLG', 'BABIP', 'BIP', 'OBP', 'OPS']), pd.DataFrame(merge, columns = ['next_game_HIT'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

clf = RandomForestClassifier()
clf.fit(x_train, y_train.values.ravel())

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [84]:
predictions = clf.predict(x_test)
predictions = [float(i) for i in predictions]
comparison = pd.DataFrame({"Real" : y_test.iloc[:,0], "Predictions" : predictions})
comparison = comparison.reset_index()
comparison = comparison.drop(['index'], axis=1)
correct = 0
for i in range(len(comparison)):
    if comparison.at[i, 'Real'] == comparison.at[i, 'Predictions']:
        correct += 1
print (correct / len(comparison))

0.5768177669607545


In [85]:
false = comparison[comparison['Real'] != comparison['Predictions']]
false = false.reset_index()
false = false.drop(['index'], axis = 1)
falsePositive = 0
for i in range(len(false)):
    if false.at[i, 'Real'] == 0:
        falsePositive += 1
print (falsePositive / len(false))

0.6606757728253055
